### 4. Working with Different Types of Data

In [0]:
df = (spark.read.format("csv")
 .option("header", "true")
 .option("inferSchema", "true")
 .load("/databricks-datasets/definitive-guide/data/retail-data/by-day/2010-12-01.csv")
)
df.printSchema()
df.count()
df.show(5)

root
 |-- InvoiceNo: string (nullable = true)
 |-- StockCode: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Quantity: integer (nullable = true)
 |-- InvoiceDate: timestamp (nullable = true)
 |-- UnitPrice: double (nullable = true)
 |-- CustomerID: double (nullable = true)
 |-- Country: string (nullable = true)

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|
|   536365| 

**NOTE About inferSchema**

Using inferSchema=false (default option) will give a dataframe where all columns are strings (StringType).

By setting inferSchema=true, Spark will automatically go through the csv file and infer the schema of each column. This requires an extra pass over the file which will result in reading a file with inferSchema set to true being slower. But in return the dataframe will most likely have a correct schema given its input.

Using inferSchema=True, there is no longer a lazy evaluation

#### 4.1 Converting to Spark Types (reminder)

In [0]:
from pyspark.sql.functions import lit
df.select("InvoiceNo", lit(5), lit("five"), lit(5.0))

Out[30]: DataFrame[InvoiceNo: string, 5: int, five: string, 5.0: double]

#### 4.2 Working with Booleans

In [0]:
from pyspark.sql.functions import col
df.filter(col("InvoiceNo") == "536365").show() # same as .where()

+---------+-----------------------------------+
|InvoiceNo|Description                        |
+---------+-----------------------------------+
|536365   |WHITE HANGING HEART T-LIGHT HOLDER |
|536365   |WHITE METAL LANTERN                |
|536365   |CREAM CUPID HEARTS COAT HANGER     |
|536365   |KNITTED UNION FLAG HOT WATER BOTTLE|
|536365   |RED WOOLLY HOTTIE WHITE HEART.     |
+---------+-----------------------------------+
only showing top 5 rows



In [0]:
df.where("InvoiceNo <> '536365'").show(5, False)

+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description                  |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+---------+---------+-----------------------------+--------+-------------------+---------+----------+--------------+
|536366   |22633    |HAND WARMER UNION JACK       |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536366   |22632    |HAND WARMER RED POLKA DOT    |6       |2010-12-01 08:28:00|1.85     |17850.0   |United Kingdom|
|536367   |84879    |ASSORTED COLOUR BIRD ORNAMENT|32      |2010-12-01 08:34:00|1.69     |13047.0   |United Kingdom|
|536367   |22745    |POPPY'S PLAYHOUSE BEDROOM    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
|536367   |22748    |POPPY'S PLAYHOUSE KITCHEN    |6       |2010-12-01 08:34:00|2.1      |13047.0   |United Kingdom|
+---------+---------+-----------------------------+--------+----

In [0]:
# instr() # position of work, begins from 1
from pyspark.sql.functions import instr
(df.withColumn("isWhite", instr(df.Description, "WHITE"))
    .select('Description', 'isWhite')
    .show(5, False)
)

# .isin()
df.where(df.StockCode.isin("DOT", "22633")).select("StockCode", "Country").show()

+---------+--------------+
|StockCode|       Country|
+---------+--------------+
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|    22633|United Kingdom|
|      DOT|United Kingdom|
|    22633|United Kingdom|
|      DOT|United Kingdom|
+---------+--------------+



Out[58]: [Row(s=2)]

#### 4.3 Working with Numbers

In [0]:
# By default, the round function rounds up if you’re exactly in between two numbers. You can round down by using the bround

# from pyspark.sql.functions import lit, round, bround


# df.select(round(lit("2.5")), bround(lit("2.5"))).show(20)



# df.stat.freqItems(["Country"]).show(1, False)

# df.select("Country").distinct().show()

StockCode_freqItems,Quantity_freqItems
"List(90214E, 20728, 20755, 21703, 22113, 22524, 22041, 72803A, 72798C, 90181B, 21756, 22694, 90206C, 20970, 21624, 90209C, 84744, 82494L, 22952, 20682, 22583, 21705, 20679, 22220, 90177E, 90214A, 22448, 90214S, 22121, 22802, 84970L, 72818, 90192, 90200C, 22910, 21380, 90211A, 21137, 35271S, 84926A, 20765, 22384, 21524, 22165, 22366, 21221, 21704, 22519, 85035C, 21967, 22114, 22909, 22900, 22447, 21577, 21877, 20726, 85034A, DOT, 84658, 21472, 22804, 22222, 72802C, 21739, 22467, 90214H, 22785, 22446, 22197, 20665, 21733, 22731, 21709, 22086, 40001, 85123A)","List(200, 128, 23, 32, 50, 600, 8, 17, 80, -1, -10, 11, 56, 47, 20, -7, 2, 5, 480, -4, 14, 432, 100, 64, 40, 13, 4, -5, 22, 16, -2, 7, 70, 384, 25, 34, 10, 1, 288, 216, 28, 252, 19, 120, 192, 60, 96, 72, 144, 36, 27, 9, 18, 48, 21, 12, 3, -6, -24, 30, 15, 33, 6, 24, -12, -3)"


In [0]:
# Convertir vers 
df.stat.crosstab("Country", "StockCode").display()


<bound method PandasConversionMixin.toPandas of DataFrame[StockCode_Quantity: string, -1: bigint, -10: bigint, -12: bigint, -2: bigint, -24: bigint, -3: bigint, -4: bigint, -5: bigint, -6: bigint, -7: bigint, 1: bigint, 10: bigint, 100: bigint, 11: bigint, 12: bigint, 120: bigint, 128: bigint, 13: bigint, 14: bigint, 144: bigint, 15: bigint, 16: bigint, 17: bigint, 18: bigint, 19: bigint, 192: bigint, 2: bigint, 20: bigint, 200: bigint, 21: bigint, 216: bigint, 22: bigint, 23: bigint, 24: bigint, 25: bigint, 252: bigint, 27: bigint, 28: bigint, 288: bigint, 3: bigint, 30: bigint, 32: bigint, 33: bigint, 34: bigint, 36: bigint, 384: bigint, 4: bigint, 40: bigint, 432: bigint, 47: bigint, 48: bigint, 480: bigint, 5: bigint, 50: bigint, 56: bigint, 6: bigint, 60: bigint, 600: bigint, 64: bigint, 7: bigint, 70: bigint, 72: bigint, 8: bigint, 80: bigint, 9: bigint, 96: bigint]>


In [0]:
from pyspark.sql.functions import monotonically_increasing_id, expr
df.select(monotonically_increasing_id().alias("id"), expr("*")).show(20)

+-----------------------------+
|monotonically_increasing_id()|
+-----------------------------+
|                            0|
|                            1|
|                            2|
|                            3|
|                            4|
|                            5|
|                            6|
|                            7|
|                            8|
|                            9|
|                           10|
|                           11|
|                           12|
|                           13|
|                           14|
|                           15|
|                           16|
|                           17|
|                           18|
|                           19|
+-----------------------------+
only showing top 20 rows



#### 4.4 Working withStrings

In [0]:
# initcap, lower, upper
from pyspark.sql.functions import initcap, lower, upper


(df.select(col("Description"),
           lower(col("Description")),
           upper(lower(col("Description"))),
           initcap(col("Description"))
          ).show(2))


(df.select(col("Description"),
           lower(("Description")),
           upper("Description"),
           initcap(col("Description"))
          ).show(2))

+--------------------+--------------------+--------------------+--------------------+
|         Description|  lower(Description)|  upper(Description)|initcap(Description)|
+--------------------+--------------------+--------------------+--------------------+
|WHITE HANGING HEA...|white hanging hea...|WHITE HANGING HEA...|White Hanging Hea...|
| WHITE METAL LANTERN| white metal lantern| WHITE METAL LANTERN| White Metal Lantern|
+--------------------+--------------------+--------------------+--------------------+
only showing top 2 rows



In [0]:
# ltrim, rtrim, rpad, lpad, trim, substring
from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim, substring
(df.select(
    ltrim(lit('   HELLO  ')).alias('ltrim'),
    rtrim(lit('   HELLO  ')).alias('rtrim'),
    substring(lit('HELLO'), 1, 3).alias('substr'),
    rpad(lit('HELLO'), 10, "x").alias('rap'),
    lpad(lit('HELLO'), 10, "x").alias('lpad')

).show(1))

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2413851590457828> in <cell line: 2>()
      1 from pyspark.sql.functions import lit, ltrim, rtrim, rpad, lpad, trim, substring
----> 2 (df.select(
      3     ltrim(lit('   HELLO  ')).alias('ltrim'),
      4     rtrim(lit('   HELLO  ')).alias('rtrim'),
      5     substring(lit('HELLO'), 1, 3).alias('substr'),

NameError: name 'df' is not defined

##### Regular Expressions

In [0]:
from pyspark.sql.functions import regexp_replace
regex_string = "BLACK|WHITE|RED|GREEN|BLUE"
df.select(
regexp_replace("Description", regex_string, "COLOR").alias("color_clean"),
"Description").show(2)


from pyspark.sql.functions import regexp_extract
extract_str = "(BLACK|WHITE|RED|GREEN|BLUE)"
df.select(
regexp_extract("Description", extract_str, 0).alias("color_clean"),
"Description").show(2)

# idx indicates which regex group to extract. An idx of 0 means matching the entire regular expression

+--------------------+--------------------+
|         color_clean|         Description|
+--------------------+--------------------+
|COLOR HANGING HEA...|WHITE HANGING HEA...|
| COLOR METAL LANTERN| WHITE METAL LANTERN|
+--------------------+--------------------+
only showing top 2 rows

+-----------+--------------------+
|color_clean|         Description|
+-----------+--------------------+
|      WHITE|WHITE HANGING HEA...|
|      WHITE| WHITE METAL LANTERN|
+-----------+--------------------+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import expr, locate
simpleColors = ["black", "white", "red", "green", "blue"]
def color_locator(column, color_string):
    return locate(color_string.upper(), column).cast("boolean").alias("is_" + color_string)
selectedColumns = [color_locator(df.Description, c) for c in simpleColors]
selectedColumns.append(expr("*")) # has to a be Column type

df.select(*selectedColumns).where(expr("is_white OR is_red"))\
.show(3, False)


selectedColumns

+--------+--------+------+--------+-------+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|is_black|is_white|is_red|is_green|is_blue|InvoiceNo|StockCode|Description                       |Quantity|InvoiceDate        |UnitPrice|CustomerID|Country       |
+--------+--------+------+--------+-------+---------+---------+----------------------------------+--------+-------------------+---------+----------+--------------+
|false   |true    |false |false   |false  |536365   |85123A   |WHITE HANGING HEART T-LIGHT HOLDER|6       |2010-12-01 08:26:00|2.55     |17850.0   |United Kingdom|
|false   |true    |false |false   |false  |536365   |71053    |WHITE METAL LANTERN               |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
|false   |true    |true  |false   |false  |536365   |84029E   |RED WOOLLY HOTTIE WHITE HEART.    |6       |2010-12-01 08:26:00|3.39     |17850.0   |United Kingdom|
+--------+------

#### 4.5 Working with Dates and Timestamps

In [0]:
from pyspark.sql.functions import current_date, current_timestamp
dateDF = spark.range(1).withColumn("today", current_date()).withColumn("now", current_timestamp())
dateDF.show(1, False)
dateDF.createOrReplaceTempView("dateTable")
dateDF.printSchema()


from pyspark.sql.functions import date_add, date_sub
dateDF.select(date_sub("today", 5), date_add("today", 5)).show(1)

+---+----------+-----------------------+
|id |today     |now                    |
+---+----------+-----------------------+
|0  |2023-04-05|2023-04-05 16:55:28.232|
+---+----------+-----------------------+

root
 |-- id: long (nullable = false)
 |-- today: date (nullable = false)
 |-- now: timestamp (nullable = false)

+------------------+------------------+
|date_sub(today, 5)|date_add(today, 5)|
+------------------+------------------+
|        2023-03-31|        2023-04-10|
+------------------+------------------+



In [0]:
from pyspark.sql.functions import to_date, to_timestamp, datediff, months_between, year, month
(df.withColumn('date', to_date('InvoiceDate'))
 .withColumn('year', year('InvoiceDate'))
 .show(2))

# EXO : une date ref, calculer monthe between

# (df.withColumn('date', to_date('InvoiceDate')).select("date").distinct().show()) # il y a une seule date dans ce df

df.withColumn('date_ref', lit('2013-10-01')).withColumn('delai', months_between('date_ref', 'InvoiceDate')).show() # in months_between function no need for col()

+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+
|InvoiceNo|StockCode|         Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|  date_ref|delai|
+---------+---------+--------------------+--------+-------------------+---------+----------+--------------+----------+-----+
|   536365|   85123A|WHITE HANGING HEA...|       6|2010-12-01 08:26:00|     2.55|   17850.0|United Kingdom|2013-10-01| 34.0|
|   536365|    71053| WHITE METAL LANTERN|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|2013-10-01| 34.0|
|   536365|   84406B|CREAM CUPID HEART...|       8|2010-12-01 08:26:00|     2.75|   17850.0|United Kingdom|2013-10-01| 34.0|
|   536365|   84029G|KNITTED UNION FLA...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|2013-10-01| 34.0|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|2010-12-01 08:26:00|     3.39|   17850.0|United Kingdom|2013-10-01| 34.0|


#### 4.6 Working with Nulls in Data

In [0]:
df.printSchema
df.filter("Description is NULL").show() # InvoiceNo = 536414
df.filter(col("Description").isNull()).show() # here, col() is needed 

# or we can do 
# df.filter(df.description.isNull()).show()




---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1445277380721950> in <cell line: 1>()
----> 1 df.printSchema
      2 df.filter("Description is NULL").show() # InvoiceNo = 536414
      3 df.filter(col("Description").isNull()).show() # here, col() is needed
      4 
      5 # or we can do

NameError: name 'df' is not defined

In [0]:
# coalesce()
# select the first non-null value from a set of columns by using the coalesce function.

from pyspark.sql.functions import coalesce
df.filter("Description is NULL").show()

df.filter("CustomerID is NULL").select("Description", "Country",coalesce("Description", "Country")).show()


+-----------------------------------+
|coalesce(Description, CustomerId)  |
+-----------------------------------+
|WHITE HANGING HEART T-LIGHT HOLDER |
|WHITE METAL LANTERN                |
|CREAM CUPID HEARTS COAT HANGER     |
|KNITTED UNION FLAG HOT WATER BOTTLE|
|RED WOOLLY HOTTIE WHITE HEART.     |
|SET 7 BABUSHKA NESTING BOXES       |
|GLASS STAR FROSTED T-LIGHT HOLDER  |
|HAND WARMER UNION JACK             |
|HAND WARMER RED POLKA DOT          |
|ASSORTED COLOUR BIRD ORNAMENT      |
|POPPY'S PLAYHOUSE BEDROOM          |
|POPPY'S PLAYHOUSE KITCHEN          |
|FELTCRAFT PRINCESS CHARLOTTE DOLL  |
|IVORY KNITTED MUG COSY             |
|BOX OF 6 ASSORTED COLOUR TEASPOONS |
|BOX OF VINTAGE JIGSAW BLOCKS       |
|BOX OF VINTAGE ALPHABET BLOCKS     |
|HOME BUILDING BLOCK WORD           |
|LOVE BUILDING BLOCK WORD           |
|RECIPE BOX WITH METAL HEART        |
+-----------------------------------+
only showing top 20 rows



In [0]:
# .drop()
# The simplest function is drop, which removes rows that contain nulls.
df.count() # 3108
df.na.drop().count() #1968
df.na.drop("any").count() # as same as the previous row
df.na.drop("all").count() # 3108 all column null

df.na.drop("any", subset=["StockCode", "description"]).count() # 3098

Out[8]: 3098

In [0]:
# .fill()


df.na.fill(5, subset=["CustomerID", "Description"]).filter(col("InvoiceNo")==536414).show()
# pay attention to result, because of data type, descripton is still null


# so the solution is
fill_cols_vals = {"CustomerID": 5, "Description" : "No Value"}
df.na.fill(fill_cols_vals).filter(col("InvoiceNo")==536414).show()

+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+
|   536414|    22139|       null|      56|2010-12-01 11:52:00|      0.0|       5.0|United Kingdom|
+---------+---------+-----------+--------+-------------------+---------+----------+--------------+



In [0]:
# .replace() : this is not about missin value but a more general replacement
df.printSchema()
df.filter(col("Description")=="").show()
from pyspark.sql.functions import col

df.replace(["United Kingdom"], ["UNKNOWN"], "Country").filter(col("InvoiceNo")==536414).show()

+---------+---------+-----------+--------+-------------------+---------+----------+-------+
|InvoiceNo|StockCode|Description|Quantity|        InvoiceDate|UnitPrice|CustomerID|Country|
+---------+---------+-----------+--------+-------------------+---------+----------+-------+
|   536414|    22139|       null|      56|2010-12-01 11:52:00|      0.0|      null|UNKNOWN|
+---------+---------+-----------+--------+-------------------+---------+----------+-------+



#### 4.7 Working with Complex Types

* Le monde ne tourne pas qu'au tour de tabular structuré
* Intresting when you do a lot webscrapping and json manipulaton

In [0]:
# structs() creates array within DataFrames
# df.selectExpr("struct(Description, InvoiceNo) as complex", "*").show()

from pyspark.sql.functions import struct

complexDF = df.select(struct("Description", "InvoiceNo").alias("complex"))

complexDF.show(5, False)

complexDF.select(col("complex").getField("Description")).show()

# if in your data there is alreay a complexe type that you want to remodeling
complexDF.select("complex.*").show()

+---------------------------------------------+
|complex                                      |
+---------------------------------------------+
|{WHITE HANGING HEART T-LIGHT HOLDER, 536365} |
|{WHITE METAL LANTERN, 536365}                |
|{CREAM CUPID HEARTS COAT HANGER, 536365}     |
|{KNITTED UNION FLAG HOT WATER BOTTLE, 536365}|
|{RED WOOLLY HOTTIE WHITE HEART., 536365}     |
+---------------------------------------------+
only showing top 5 rows



**Arrays** List ????

To define arrays, let’s work through a use case. With our current data, our objective is to take every single word in our Description column and convert that into a row in our DataFrame.

In [0]:
from pyspark.sql.functions import split
df.select(col("Description")).show(2)

df.select(split("Description", " ")).show(2)

+--------------------+
|         Description|
+--------------------+
|WHITE HANGING HEA...|
| WHITE METAL LANTERN|
+--------------------+
only showing top 2 rows

+-------------------------+
|split(Description,  , -1)|
+-------------------------+
|     [WHITE, HANGING, ...|
|     [WHITE, METAL, LA...|
+-------------------------+
only showing top 2 rows



In [0]:
from pyspark.sql.functions import split, explode

df.select(split("Description", " ")).show(2, False)


# Opposite of explode collect_set (or collect_list)

df.select("Description", "InvoiceNo",explode(split("Description", " "))).show(6, False)
# .count() # 14414 dupliquer les lignes



Out[40]: 14414

In [0]:
# create_map : key -> value
from pyspark.sql.functions import create_map

df.select(create_map("InvoiceNo", "Description").alias("complex_map")).show(5, False)

df.select(create_map("InvoiceNo", "Description").alias("complex_map")).selectExpr("complex_map[536365]").show(5, False)

# you can also explode map type, which will turn them into columns
# df.select(create_map("InvoiceNo", "Description").alias("complex_map")).withColumn("Explode", explode(col("complex_map"))).show(5, False) # this does not work, try to find why

df.select(create_map("InvoiceNo", "Description").alias("complex_map")).selectExpr("explode(complex_map)").show(5, False)

+--------------------+------+
|                 key| value|
+--------------------+------+
|WHITE HANGING HEA...|536365|
| WHITE METAL LANTERN|536365|
+--------------------+------+
only showing top 2 rows



In [0]:
# array_contains
# intresting for complex join

from pyspark.sql.functions import array_contains
df.select("Description",
          split("Description", " ").alias("split"),
          array_contains(split("Description", " "), "WHITE").alias('white')).show(2, False)

#### Working with JSON

In [0]:
from pyspark.sql.functions import from_json, to_json
from pyspark.sql.types import *
df.selectExpr("(InvoiceNo, Description) as myStruct")\
.select(to_json(col("myStruct")).alias("newJSON"))\
.select(from_json(col("newJSON"), parseSchema), col("newJSON")).show(2)

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-1445277380721957> in <cell line: 3>()
      3 df.selectExpr("(InvoiceNo, Description) as myStruct")\
      4 .select(to_json(col("myStruct")).alias("newJSON"))\
----> 5 .select(from_json(col("newJSON")), col("newJSON")).show(2)

TypeError: from_json() missing 1 required positional argument: 'schema'

#### User-Defined Functions

In [0]:
# function python
def power3(double_value):
    return double_value ** 3
power3(2.0)

Out[77]: 8.0

Now that we’ve created these functions and tested them, we need to register them with Spark so that we can use them on all of our worker machines. Spark will serialize the function on the driver and transfer it over the network to all executor processes. This happens regardless of language. When you use the function, there are essentially two different things that occur. If the function is written in Scala or Java, you can use it within the Java Virtual Machine (JVM). This means that there will be little performance penalty aside from the fact that you can’t take advantage of code generation capabilities that Spark has for built-in functions.

If the function is written in Python, something quite different happens. Spark starts a Python process on the worker, serializes all of the data to a format that Python can understand (remember, it was in the JVM earlier), executes the function row by row on that data in the Python process, and then finally returns the results of the row operations to the JVM and Spark.

**why pyspark functions works with performance but not Python UDF function?**

https://stackoverflow.com/questions/38296609/spark-functions-vs-udf-performance

Spark DataFrame is natively a JVM structure and standard access methods are implemented by simple calls to Java API. UDF from the other hand are implemented in Python and require moving data back and forth.

While PySpark in general requires data movements between JVM and Python, in case of low level RDD API it typically doesn't require expensive serde activity. Spark SQL adds additional cost of serialization and serialization as well cost of moving data from and to unsafe representation on JVM. The later one is specific to all UDFs (Python, Scala and Java) but the former one is specific to non-native languages.

Unlike UDFs, **Spark SQL functions** such as pyspark function operate directly on JVM and typically are well integrated with both Catalyst and Tungsten. It means these can be optimized in the execution plan and most of the time can benefit from codgen and other Tungsten optimizations. Moreover these can operate on data in its "native" representation.

In [0]:
# in Python

# First, we need to register the function with Spark to make it available as a DataFrame function, so that we can use them on all of our executor processes machines.Spark will serialize the function on the driver and transfer it over the network to all executor processes with udf().
from pyspark.sql.functions import udf
power3udf= udf(power3)


ExampleDF = spark.range(5).withColumnRenamed('id', 'num')
ExampleDF.show()

ExampleDF.select(power3udf("num"), "num").show()

+-----------+
|power3(num)|
+-----------+
|          0|
|          1|
|          8|
|         27|
|         64|
+-----------+



*Comment about import elements*
### 5. Aggregations

In [0]:
df_Schema = df.schema

df = (spark.read.format("csv")
      .option("header", "true")
      .option("inferSchema", "true") # avoir to use this, prefer lazy evaluation
#       .schema(df_Schema) # tu ne peux pas utiliser cela directement car dans les data sur la colonne date il y a un prob, il arrive pas a assiger type timestamps
      .load("/databricks-datasets/definitive-guide/data/retail-data/all/*.csv")
      .where("InvoiceDate like '%2010%'") # 42481
#       .where("CustomerId IS NOT NULL")
      .filter(col("InvoiceDate").like("%2010%")) # 42481
      .coalesce(5))
      

df.rdd.getNumPartitions() # 8 by default and 5 after .coalesce(5)
df.cache()
df.createOrReplaceTempView("dfTable")

df.show(5)
df.schema

df.count() #541909 => 42481 after filter
# df.where("CustomerId IS NOT NULL").count() # 406829

Out[46]: 0

**NOTE : Spark – Difference between Cache and Persist**

Spark Cache and persist are optimization techniques for iterative and interactive Spark applications to improve the performance of the jobs or applications.

https://sparkbyexamples.com/spark/spark-difference-between-cache-and-persist/

#### 5.1 Aggregation Functions
* combines with select() function but not selectExpr()
* df.selectExpr("stockCode").show(5) ==> OK
* df.selectExpr(countDistinct("stockCode")).show(5) ==> NOT OK, use select()

In [0]:
# count : The first function worth going over is count, except in this example it will perform as a transformation instead of an action
from pyspark.sql.functions import count
df.select(count("stockCode")).show() # je écris, vous commentez. ici count action or transformation. count() is a function here

df.select(count("*")).show() # 406829

In [0]:
# countDistinct
from pyspark.sql.functions import countDistinct
df.select(countDistinct("stockCode")).show() # 4070

In [0]:
# approx_count_distinct
# Often, we find ourselves working with large datasets and the exact distinct count is irrelevant. There are times when an approximation to a certain degree of accuracy will work just fine

from pyspark.sql.functions import approx_count_distinct
df.select(approx_count_distinct("stockCode", 0.05)).show() # maximum relative standard deviation allowed (default = 0.05)

# approx_count_distinct took another parameter with which you can specify the maximum estimation error allowed. In this case, we specified a rather large error andthus receive an answer that is quite far off but does complete more quickly than countDistinct. You will see much greater performance gains with larger datasets.

+--------------------------------+
|approx_count_distinct(StockCode)|
+--------------------------------+
|                            3804|
+--------------------------------+



In [0]:
# first and last
from pyspark.sql.functions import first, last
df.select(first("StockCode"), last("StockCode")).show()

In [0]:
# min and max

from pyspark.sql.functions import min, max
df.select(min("Quantity"), max("Quantity")).show()

In [0]:
# sum
from pyspark.sql.functions import sum
df.select(sum("Quantity")).show() # 342228

spark.sql("SELECT sum(Quantity) FROM dfTable").show() # 342228

In [0]:
# sumDistinct
from pyspark.sql.functions import sumDistinct
df.select(sumDistinct("Quantity")).show() # 22884

In [0]:
# avg
from pyspark.sql.functions import avg, sum
df.show(5)
df.select(avg("Quantity")).show()

# EXO, calculer montant moyen par trasaction
from pyspark.sql.functions import expr
(
    df.select(
    sum(expr("Quantity * UnitPrice")).alias("Price"), 
    count("Quantity").alias("Total_Transactions"))
 .selectExpr("Price/Total_Transactions as avg_price").show(4)
 )

In [0]:
# Variance and Standard Deviation
from pyspark.sql.functions import var_pop, stddev_pop, var_samp, stddev_samp
(df.select(var_pop("Quantity"), 
          var_samp("Quantity"),
         stddev_pop("Quantity"),
         stddev_samp("Quantity")).show())

In [0]:
# skewness and kurtosis
# Both measurements of extreme points in your data. Skewness measures the asymmetry of the values in your data around the mean, whereas kurtosis is a measure of the tail of data. These are both relevant specifically when modeling your data as a probability distribution of a random variable
from pyspark.sql.functions import skewness, kurtosis
df.select(skewness("Quantity"), kurtosis("Quantity")).show()

+-------------------+------------------+
| skewness(Quantity)|kurtosis(Quantity)|
+-------------------+------------------+
|-0.2640755761052562|119768.05495536952|
+-------------------+------------------+



In [0]:
# Covariance and Correlation
# corr : correlation
# covar_pop : the population covariance
# covar_samp : sample covariance
from pyspark.sql.functions import corr, covar_pop, covar_samp
df.select(corr("InvoiceNo","Quantity"), covar_pop("InvoiceNo","Quantity"), covar_samp("InvoiceNo","Quantity")).show()

+-------------------------+------------------------------+-------------------------------+
|corr(InvoiceNo, Quantity)|covar_pop(InvoiceNo, Quantity)|covar_samp(InvoiceNo, Quantity)|
+-------------------------+------------------------------+-------------------------------+
|     4.912186085635685E-4|            1052.7260778741693|             1052.7280543902734|
+-------------------------+------------------------------+-------------------------------+



In [0]:
# Aggregating to Complex Types, opposite is explode()

# The collect_list() operation is not responsible for unifying the array list. It fills all the elements by their existing order and does not remove any of the duplications. On the other hand, the collect_set() operation does eliminate the duplicates; however, it cannot save the existing order of the items in the array.

from pyspark.sql.functions import collect_list, collect_set

df.select(collect_list("Country"), collect_set("Country")).show() # we can also use select()
# df.agg(collect_list("Country"), collect_set("Country")).show()

+---------------------+--------------------+
|collect_list(Country)|collect_set(Country)|
+---------------------+--------------------+
| [United Kingdom, ...|[Portugal, Italy,...|
+---------------------+--------------------+



#### 5.2 Grouping

df.grouBy.unique_aggregation_function()

df.grouBy.agg(function_1, function_2)

In [0]:
# POSER LA QUESTION : count here is a transformation or a action
df.groupBy("Country", "").count().show(5) # note that count() is a transformation not a action

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1077835294916368> in <cell line: 1>()
----> 1 df.groupBy("Country", "Quantity").count().show(5)

NameError: name 'df' is not defined

In [0]:
# Grouping with Maps
df.groupBy("Country").agg(sum("Quantity"), 
                          count("Quantity")).show(10)

+---------+---------------+-------------+
|  Country|count(Quantity)|sum(Quantity)|
+---------+---------------+-------------+
|   Sweden|            462|        35637|
|Singapore|            229|         5234|
|  Germany|           9495|       117448|
|   France|           8557|       110480|
|   Greece|            146|         1556|
+---------+---------------+-------------+
only showing top 5 rows



In [0]:

df.groupBy("Country").agg(avg("Quantity"), stddev_pop("Quantity")).show(2)

+---------+-----------------+--------------------+
|  Country|    avg(Quantity)|stddev_pop(Quantity)|
+---------+-----------------+--------------------+
|   Sweden|77.13636363636364|  128.75197143945962|
|Singapore|22.85589519650655|  27.682246020195702|
+---------+-----------------+--------------------+
only showing top 2 rows



#### 5.3 Window Functions
##### Example 1

In [0]:
# Copier coller les code depuis codeshare
from pyspark.sql import Row
simpleData = [Row("James", "Sales", 3000),
    Row("Michael", "Sales", 4600),
    Row("Robert", "Sales", 4100),
    Row("Maria", "Finance", 3000),
    Row("James", "Sales", 3000),
    Row("Scott", "Finance", 3300),
    Row("Jen", "Finance", 3900),
    Row("Jeff", "Marketing", 3000),
    Row("Kumar", "Marketing", 2000),
    Row("Saif", "Sales", 4100)]

# attention a toDF pour renommer
data = spark.createDataFrame(simpleData).toDF("employee_name", "department", "salary")

data.printSchema()
data.show()

+-------------+----------+------+
|employee_name|department|salary|
+-------------+----------+------+
|        James|     Sales|  3000|
|      Michael|     Sales|  4600|
|       Robert|     Sales|  4100|
|        Maria|   Finance|  3000|
|        James|     Sales|  3000|
|        Scott|   Finance|  3300|
|          Jen|   Finance|  3900|
|         Jeff| Marketing|  3000|
|        Kumar| Marketing|  2000|
|         Saif|     Sales|  4100|
+-------------+----------+------+



%md
* The first step to a window function is to create a window specification, with *partitionby* (is unrelated to the partitioning scheme concept)
* The ordering determines the ordering within a given partition. The frame specification (the rowsBetween statement) states which rows will be included in the frame based on its reference to the current input row.

In [0]:
from pyspark.sql.window import Window
windowSpec  = Window.partitionBy("department").orderBy("salary")

type(windowSpec)

Out[4]: pyspark.sql.window.WindowSpec

In [0]:
from pyspark.sql.functions import row_number # syntaxe : function().over(partition)
data.withColumn("row_number",row_number().over(windowSpec)).show()

In [0]:
from pyspark.sql.functions import rank, dense_rank, cume_dist, lag, lead, max, min
# pay attention, you must import max and min

(data
 .withColumn("rank", rank().over(windowSpec))
 .withColumn("dense_rank", dense_rank().over(windowSpec))
 .withColumn("cume_dist", cume_dist().over(windowSpec))
 .withColumn("lag",lag("salary",1).over(windowSpec))
 
 .withColumn("lead", lead("salary",1).over(windowSpec))
 .withColumn("MINIMUM",min("salary").over(windowSpec))
).show()

---------------------------------------------------------------------------
TypeError                                 Traceback (most recent call last)
<command-1077835294916387> in <cell line: 5>()
      8  .withColumn("cume_dist",cume_dist().over(windowSpec))
      9  .withColumn("lag",lag("salary",1).over(windowSpec))
---> 10  .withColumn("max", max(col("salary")).over(windowSpecAgg))
     11  .withColumn("lead",lead("salary",2).over(windowSpec))# previous row
     12  .show()

/databricks/spark/python/pyspark/sql/column.py in __iter__(self)
    565 
    566     def __iter__(self) -> None:
--> 567         raise TypeError("Column is not iterable")
    568 
    569     # string methods

TypeError: Column is not iterable

##### Example 2

In [0]:
from pyspark.sql.functions import col, to_date
dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
dfWithDate.createOrReplaceTempView("dfWithDate")

from pyspark.sql.window import Window
from pyspark.sql.functions import desc
windowSpec = Window\
    .partitionBy("CustomerId", "date")\
    .orderBy(desc("Quantity"))\
    .rowsBetween(Window.unboundedPreceding, Window.currentRow)

from pyspark.sql.functions import max
maxPurchaseQuantity = max(col("Quantity")).over(windowSpec)

from pyspark.sql.functions import dense_rank, rank
purchaseDenseRank = dense_rank().over(windowSpec)
purchaseRank = rank().over(windowSpec)

dfWithDate.where("CustomerId IS NOT NULL").orderBy("CustomerId")\
.select(
col("CustomerId"),
col("date"),
col("Quantity"),
purchaseRank.alias("quantityRank"),
purchaseDenseRank.alias("quantityDenseRank"),
maxPurchaseQuantity.alias("maxPurchaseQuantity")).show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-2413851590457802> in <cell line: 2>()
      1 from pyspark.sql.functions import col, to_date
----> 2 dfWithDate = df.withColumn("date", to_date(col("InvoiceDate"), "MM/d/yyyy H:mm"))
      3 dfWithDate.createOrReplaceTempView("dfWithDate")
      4 
      5 from pyspark.sql.window import Window

NameError: name 'df' is not defined

#### 5.4 Grouping Sets

In [0]:
# Pivot
from pyspark.sql.functions import to_date, hour
dfWithDate = df.withColumn('hour', hour("InvoiceDate"))
dfWithDate.groupBy("hour").pivot("Country").sum("Quantity").orderBy("hour").display()


pivoted.where("hour > '12'").select("hour" ,"United Kingdom", "Norway").show()

In [0]:
# Rollups (optionnal)
# A rollup is a multidimensional aggregation that performs a variety of group-by style calculations for us
rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
  .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
  .orderBy("Date")
rolledUpDF.show()

---------------------------------------------------------------------------
NameError                                 Traceback (most recent call last)
<command-1077835294916372> in <cell line: 3>()
      1 # Rollups
      2 # A rollup is a multidimensional aggregation that performs a variety of group-by style calculations for us
----> 3 rolledUpDF = dfNoNull.rollup("Date", "Country").agg(sum("Quantity"))\
      4   .selectExpr("Date", "Country", "`sum(Quantity)` as total_quantity")\
      5   .orderBy("Date")

NameError: name 'dfNoNull' is not defined

In [0]:
# Cube (optionnal)

# A cube takes the rollup to a level deeper. Rather than treating elements hierarchically, a cube does the same thing across all dimensions

from pyspark.sql.functions import sum
dfNoNull.cube("Date", "Country").agg(sum(col("Quantity")))\
.select("Date", "Country", "sum(Quantity)").orderBy("Date").show(100)

+----------+---------------+-------------+
|      Date|        Country|sum(Quantity)|
+----------+---------------+-------------+
|      null|        Finland|         1254|
|      null|      Lithuania|          652|
|      null|         Poland|          140|
|      null|        Iceland|          319|
|      null|        Germany|         6723|
|      null|         France|         4978|
|      null|          Italy|          293|
|      null|          Japan|         4093|
|      null| United Kingdom|       298101|
|      null|         Sweden|         3714|
|      null|        Denmark|          454|
|      null|           null|       342228|
|      null|         Cyprus|          917|
|      null|        Austria|            3|
|      null|      Australia|          454|
|      null|         Norway|         3582|
|      null|           EIRE|         5381|
|      null|          Spain|          867|
|      null|        Bahrain|           54|
|      null|         Israel|          -56|
|      null

### 6. Joins
#### 6.1 Join Types

* Inner joins (keep rows with keys that exist in the left and right datasets)
* Outer joins (keep rows with keys in either the left or right datasets)
* Left outer joins (keep rows with keys in the left dataset)
* Right outer joins (keep rows with keys in the right dataset)
* Left semi joins (keep the rows in the left, and only the left, dataset where the key appears in the right dataset)
* Left anti joins (keep the rows in the left, and only the left, dataset where they do not appear in the right dataset)

* Natural joins (perform a join by implicitly matching the columns between the two datasets with the same names)
* Cross (or Cartesian) joins (match every row in the left dataset with every row in the right dataset)

In [0]:
# paste through codeshare

person = spark.createDataFrame([
(0, "Bill Chambers", 0, [100]),
(1, "Matei Zaharia", 1, [500, 250, 100]),
(2, "Michael Armbrust", 1, [250, 100])])\
.toDF("id", "name", "graduate_program", "spark_status")
graduateProgram = spark.createDataFrame([
(0, "Masters", "School of Information", "UC Berkeley"),
(2, "Masters", "EECS", "UC Berkeley"),
(1, "Ph.D.", "EECS", "UC Berkeley")])\
.toDF("id", "degree", "department", "school")
sparkStatus = spark.createDataFrame([
(500, "Vice President"),
(250, "PMC Member"),
(100, "Contributor")])\
.toDF("id", "status")

person.show()
graduateProgram.show()
sparkStatus.show()

+---+----------------+----------------+---------------+
| id|            name|graduate_program|   spark_status|
+---+----------------+----------------+---------------+
|  0|   Bill Chambers|               0|          [100]|
|  1|   Matei Zaharia|               1|[500, 250, 100]|
|  2|Michael Armbrust|               1|     [250, 100]|
+---+----------------+----------------+---------------+

+---+-------+--------------------+-----------+
| id| degree|          department|     school|
+---+-------+--------------------+-----------+
|  0|Masters|School of Informa...|UC Berkeley|
|  2|Masters|                EECS|UC Berkeley|
|  1|  Ph.D.|                EECS|UC Berkeley|
+---+-------+--------------------+-----------+

+---+--------------+
| id|        status|
+---+--------------+
|500|Vice President|
|250|    PMC Member|
|100|   Contributor|
+---+--------------+



In [0]:
# join key definition
joinExpression = person["graduate_program"] == graduateProgram["id"]
person.join(graduateProgram, joinExpression, how = "left_outer").show()

graduateProgram.join(person, joinExpression, how="left_semi").show()
graduateProgram.join(person, joinExpression, how="left_anti").show()

In [0]:
# join key definition
joinExpression = person["graduate_program"] == graduateProgram['id']

person.join(graduateProgram, joinExpression).show() # inner is the default option


# "inner" "outer", "left_outer" "right_outer" "left_semi", "left_anti" "cross"
person.join(graduateProgram, joinExpression, how = "left_outer").show()



# left_semi example : the part for with there is corresponding key from left table
graduateProgram.join(person, joinExpression, how="left_semi").show()

# left_anti example : the part for with there is no corresponding key from left table
graduateProgram.join(person, joinExpression, how="left_anti").show()



# person.join(graduateProgram, joinExpression).explain() # on ne voit pas de broadcast en python mais seulement en scala
# person.rdd.getNumPartitions()
# graduateProgram.rdd.getNumPartitions()

== Physical Plan ==
AdaptiveSparkPlan isFinalPlan=false
+- SortMergeJoin [graduate_program#20L], [id#35L], Inner
   :- Sort [graduate_program#20L ASC NULLS FIRST], false, 0
   :  +- Exchange hashpartitioning(graduate_program#20L, 200), ENSURE_REQUIREMENTS, [plan_id=211]
   :     +- Project [_1#10L AS id#18L, _2#11 AS name#19, _3#12L AS graduate_program#20L, _4#13 AS spark_status#21]
   :        +- Filter isnotnull(_3#12L)
   :           +- Scan ExistingRDD[_1#10L,_2#11,_3#12L,_4#13]
   +- Sort [id#35L ASC NULLS FIRST], false, 0
      +- Exchange hashpartitioning(id#35L, 200), ENSURE_REQUIREMENTS, [plan_id=212]
         +- Project [_1#27L AS id#35L, _2#28 AS degree#36, _3#29 AS department#37, _4#30 AS school#38]
            +- Filter isnotnull(_1#27L)
               +- Scan ExistingRDD[_1#27L,_2#28,_3#29,_4#30]




#### 6.2 Joins on Complex Types

In [0]:
from pyspark.sql.functions import expr
(person
 .withColumnRenamed('id', 'person_id') # old_name, new_name, you can show first without this line, name is ambigus
 .join(sparkStatus, expr("array_contains(spark_status, id)"))).show()

+--------+----------------+----------------+---------------+---+--------------+
|personId|            name|graduate_program|   spark_status| id|        status|
+--------+----------------+----------------+---------------+---+--------------+
|       0|   Bill Chambers|               0|          [100]|100|   Contributor|
|       1|   Matei Zaharia|               1|[500, 250, 100]|500|Vice President|
|       1|   Matei Zaharia|               1|[500, 250, 100]|250|    PMC Member|
|       1|   Matei Zaharia|               1|[500, 250, 100]|100|   Contributor|
|       2|Michael Armbrust|               1|     [250, 100]|250|    PMC Member|
|       2|Michael Armbrust|               1|     [250, 100]|100|   Contributor|
+--------+----------------+----------------+---------------+---+--------------+



#### Handling Duplicate Column Names

In [0]:
gradProgram3 = graduateProgram.withColumnRenamed("id", "grad_id")
joinExpr = person["graduate_program"] == gradProgram3["grad_id"]
person.join(gradProgram3, joinExpr).show()

person.join(gradProgram3, joinExpr).select("grad_id").show()

+-------+
|grad_id|
+-------+
|      0|
|      1|
|      1|
+-------+



In [0]:
gradProgramDupe = graduateProgram.withColumnRenamed("id", "graduate_program")
gradProgramDupe.col[]"graduate_program") === person.col(
"graduate_program")


#### How Spark Performs Joins